In [22]:
from typing import TypedDict, List
import random
from langgraph.graph import StateGraph, START, END

In [2]:
!pip install -qU langchain-anthropic langgraph tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 11.4 MB/s eta 0:00:00


In [35]:
class AgentState(TypedDict):
  player_name: str
  guesses: List[int]
  attempts: int
  target: int
  lower_bound: int
  upper_bound: int

In [38]:
def setup(state: AgentState) -> AgentState:
  state["attempts"] = 0
  state["guesses"] = []
  state["lower_bound"] = 1
  state["upper_bound"] = 20
  state["target"] = random.randint(state["lower_bound"], state["upper_bound"])

  return state

def guess(state: AgentState) -> AgentState:
  state["attempts"] += 1
  guess = random.randint(state["lower_bound"], state["upper_bound"])
  print(f"Im guessing {guess}")
  if guess > state["target"]:
    print("guess is greater")
    state["upper_bound"] = guess
  elif guess < state["target"]:
    print("guess is lower")
    state["lower_bound"] = guess
  state["guesses"].append(guess)

  return state

def should_continue(state: AgentState):
  if state["guesses"][-1] == state["target"]:
    print("WIN")
    return "exit"
  if state["attempts"] >= 7:
    print("LOSE")
    return "exit"

  return "loop"

In [40]:
graph = StateGraph(AgentState)

graph.add_node("setup", setup)
graph.add_node("guess", guess)
graph.add_edge("setup","guess")

graph.add_conditional_edges(
    "guess",
    should_continue,
    {
        "loop": "guess",
        "exit": END
    }
)

graph.set_entry_point("setup")
app = graph.compile()


In [43]:
app.invoke({"name": "Adolfo", "number": [], "target": -1})

Im guessing 19
guess is greater
Im guessing 17
guess is greater
Im guessing 13
guess is greater
Im guessing 1
guess is lower
Im guessing 8
guess is greater
Im guessing 2
WIN


{'guesses': [19, 17, 13, 1, 8, 2],
 'attempts': 6,
 'target': 2,
 'lower_bound': 1,
 'upper_bound': 8}